Name: **Pranav Bhagwat**<br>
Div: **BE-9**<br>
Roll no: **43161**<br>
Title: **Assignment 6: Object detection using Transfer Learning of CNN architectures**<br>

In [10]:
import torch
import torchvision
import torch.nn as nn  # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim  # For all Optimization algorithms, SGD, Adam, etc.
import torch.nn.functional as F  # All functions that don't have any parameters
from torch.utils.data import (
    DataLoader,
)  # Gives easier dataset managment and creates mini batches
import torchvision.datasets as datasets  # Has standard datasets we can import in a nice way
import torchvision.transforms as transforms  # Transformations we can perform on our dataset

In [24]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
num_classes = 10
learning_rate = 1e-3
batch_size = 1024
num_epochs = 50

In [12]:
# Simple Identity class that let's input pass without changes
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()

    def forward(self, x):
        return x

### 1. Load in a pretrained model (VGG16)

In [13]:
model = torchvision.models.vgg16(pretrained=True)

In [14]:
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

### 2. Freezing parameters in model's lower layers

In [15]:
# If you want to do finetuning then set requires_grad = False
for param in model.parameters():
    param.requires_grad = False

In [19]:
## Freezing the average pool layer of the model and add a custom classifier
model.avgpool = Identity()

### 3. Add custom classifier with several layers of trainable parameters to mode

In [18]:
model.classifier = nn.Sequential(
    nn.Linear(512, 100), nn.ReLU(), 
    nn.Linear(100, num_classes)
)
model.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

### 4. Train classifier layers on training data available for task


In [17]:
# Load Data
train_dataset = datasets.CIFAR10(
    root="dataset/", train=True, transform=transforms.ToTensor(), download=True
)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting dataset/cifar-10-python.tar.gz to dataset/


In [20]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [25]:
# Train Network
for epoch in range(num_epochs):
    losses = []

    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        losses.append(loss.item())
        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

    print(f"Cost at epoch {epoch} is {sum(losses)/len(losses):.5f}")

Cost at epoch 0 is 1.07481
Cost at epoch 1 is 1.06143
Cost at epoch 2 is 1.04898
Cost at epoch 3 is 1.03931
Cost at epoch 4 is 1.02822
Cost at epoch 5 is 1.02065
Cost at epoch 6 is 1.01188
Cost at epoch 7 is 1.00401
Cost at epoch 8 is 0.99593
Cost at epoch 9 is 0.98736
Cost at epoch 10 is 0.98071
Cost at epoch 11 is 0.97320
Cost at epoch 12 is 0.96478
Cost at epoch 13 is 0.95784
Cost at epoch 14 is 0.94961
Cost at epoch 15 is 0.94268
Cost at epoch 16 is 0.93464
Cost at epoch 17 is 0.92747
Cost at epoch 18 is 0.92085
Cost at epoch 19 is 0.91396
Cost at epoch 20 is 0.90671
Cost at epoch 21 is 0.89789
Cost at epoch 22 is 0.89123
Cost at epoch 23 is 0.88446
Cost at epoch 24 is 0.87809
Cost at epoch 25 is 0.87101
Cost at epoch 26 is 0.86366
Cost at epoch 27 is 0.85622
Cost at epoch 28 is 0.84890
Cost at epoch 29 is 0.84319
Cost at epoch 30 is 0.83885
Cost at epoch 31 is 0.83068
Cost at epoch 32 is 0.82383
Cost at epoch 33 is 0.81861
Cost at epoch 34 is 0.81195
Cost at epoch 35 is 0.80578
Co

### 5. Checking accuracy and fine tuning if required.

In [26]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on test data")

    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(
            f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}"
        )

    model.train()

In [27]:
check_accuracy(train_loader, model)

Checking accuracy on training data
Got 37926 / 50000 with accuracy 75.85
